In [97]:
import plotly.graph_objs as go
from ipywidgets import Video, Layout, VBox, HBox, HTML, widgets

import numpy as np
import pandas as pd

task2idx_dict = {
    "vself": 0,
    "vmin": 1,
    "vmax": 2,
    "headneck": 3,
    "dtspeech": 4,
    "dtmath": 5,
    "dtcarry": 6,
    "ec": 7
}

pheno2idx_dict = {
    "ataxia": 0,
    "episodic": 1,
    "hs": 2,
    "hypokinetic": 3,
    "normal": 4,
    "nph": 5,
    "paretic": 6,
    "phobic": 7,
    "ppv": 8,
    "psychogenic": 9,
    "sensory ataxis": 10,
    "spastic": 11,
    "suspectnph": 12
}

idx2pheno_dict = {v: k for k, v in pheno2idx_dict.items()}

idx2task_dict = {v: k for k, v in task2idx_dict.items()}

def idx2task(idx):
    return idx2task_dict[idx]

def idx2pheno(idx):
    return idx2pheno_dict[idx]

def tick_val_text_tasks():
    vals = list(idx2task_dict.keys())
    texts = [idx2task(i) for i in vals]
    return (vals, texts)

def tick_val_text_phenos():
    vals = list(idx2pheno_dict.keys())
    texts = [idx2pheno(i) for i in vals]
    return (vals, texts)

num_points = 341
motion_z_umap = np.load("data/motion_z_umap.npy")[:num_points, ]
x, y = motion_z_umap[:, 0], motion_z_umap[:, 1]
tasks = np.load("data/tasks_labels.npy")[:num_points, ].astype(np.int)
phenos = np.load("data/pheno_labels.npy")[:num_points, ].astype(np.int)
tasks_text = ["{}".format(idx2task(i))  for i in tasks]
phenos_text = ["{}".format(idx2pheno(i))  for i in phenos]
print("x's shape = {}\ny's shape = {}\ntasks's shape = {}\nphenos's shape = {}\n".format(
        x.shape, y.shape, tasks.shape, phenos.shape
        )
     )
df_info = pd.DataFrame({"Tasks":tasks_text, "Phenotypes":phenos_text})




x's shape = (341,)
y's shape = (341,)
tasks's shape = (341,)
phenos's shape = (341,)



In [98]:
# Load video data and widget
video_data = dict()
for i in range(x.shape[0]):
    with open("data/videos/compare/compare_{}.mp4".format(i), "rb") as f:
        b = f.read()
        video_data[i] = b
video_widget = Video(
    value = video_data[0],
    layout=Layout(height='252px', width='400px')
)

# Set HTML widget for data frame
details = HTML(
    value = df_info.iloc[0].to_frame().to_html()
)

# Dropdown menu for plotting type ("tasks" or "phenotypes")
plot_type_widget = widgets.Dropdown(
    options=["Tasks", "Phenotypes"],
    value="Tasks",
    description="Plot Types:",
)


In [99]:
def scatter_plot_type(scatter, plot_type):
    
    if plot_type == "tasks":
        target_text = tasks_text
        target_tick_func = tick_val_text_tasks
        target_color = tasks
    elif plot_type == "phenos":
        target_text = phenos_text
        target_tick_func = tick_val_text_phenos
        target_color = phenos
        
    scatter.text = target_text
    scatter.marker.color = target_color
    scatter.marker.colorbar = dict(
        title = plot_type,
        tickvals = target_tick_func()[0],
        ticktext = target_tick_func()[1],
        ticks = 'outside'
    )
    scatter.marker.colorscale = "Jet"
    

def hover_fn(trace, points, state):
    
    # Fetch the hovered point
    ind = points.point_inds[0]
    
    # Update entry in shown dataframe
    details.value = df_info.iloc[ind].to_frame().to_html()
    
    # Update image widget
    video_widget.value = video_data[ind]

def plot_type_response(change):
    if plot_type_widget.value == "Tasks":
        scatter_plot_type(scatter, "tasks")
    if plot_type_widget.value == "Phenotypes":
        scatter_plot_type(scatter, "phenos")
        
fig = go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=x,
            y=y,
            mode='markers',
        )
    ],
)

fig.layout.title = 'Latent space visualization'
fig.layout.titlefont.size = 12
fig.layout.xaxis.title = "x"
fig.layout.yaxis.title = "y"
fig.layout.autosize = False
fig.layout.width, fig.layout.height = 500, 600
fig.layout.hovermode = 'closest'

scatter = fig.data[0]
scatter.hoverinfo = "text"
scatter_plot_type(scatter, "tasks")
scatter.marker.opacity = 1
scatter.marker.size = 4


scatter.on_hover(hover_fn)
plot_type_widget.observe(plot_type_response, names="value")

HBox([fig,
      VBox([plot_type_widget, video_widget, details])])

    'data': [{'hoverinfo': 'text',
              'marker': {'color': array([2, 3…

In [2]:

colorbar_dict = dict(
    title = "tasks",
    tickvals = tick_val_text()[0],
    ticktext = tick_val_text()[1],
    ticks = 'outside'
)

trace0 = go.Scatter(
    x=x,
    y=y,
    mode="markers"
)

layout = go.Layout(
    autosize=False,
    width=800,
    height=800,
)


f = go.FigureWidget(data=[trace0], layout=layout)

scatter = f.data[0]

scatter.marker.color = tasks
scatter.marker.colorscale = 'Jet'
scatter.marker.colorbar = colorbar_dict
scatter.marker.size = [5] * 4096
f.layout.hovermode = 'closest'


# create our callback function
def update_point(trace, points, selector):
    
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#bae2be'
        s[i] = 20
        scatter.marker.color = c
        scatter.marker.size = s


scatter.on_click(update_point)

f

FigureWidget({
    'data': [{'marker': {'color': array([2, 3, 3, ..., 1, 1, 1]),
                         'col…